<a href="https://colab.research.google.com/github/mjmousavi97/Graph-Neural-Networks/blob/main/09_Graph_classification/src/graph_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00


In [1]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [2]:
dataset = TUDataset(root='.', name='PROTEINS').shuffle()

In [3]:
print(f"dataset: {dataset}")
print(f"Number of Graphs: {len(dataset)}")
print(f"Number of Nodes: {dataset[0].num_nodes}")
print(f"Number of Features: {dataset.num_features}")
print(f"Number of Classes: {dataset.num_classes}")

dataset: PROTEINS(1113)
Number of Graphs: 1113
Number of Nodes: 18
Number of Features: 3
Number of Classes: 2


In [4]:
train_dataset = dataset[:int(len(dataset) * 0.8)]
val_dataset = dataset[int(len(dataset) * 0.8):int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of validation graphs: {len(val_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 890
Number of validation graphs: 111
Number of test graphs: 112


In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("\nTrain Loader:")
for i, batch in enumerate(train_loader):
    print(f" - batch{i+1}: {batch}")

print("\nValidation Loader:")
for i, batch in enumerate(val_loader):
    print(f" - batch{i+1}: {batch}")

print("\nTest Loader")
for i, batch in enumerate(test_loader):
    print(f" - batch{i+1}: {batch}")


Train Loader:
 - batch1: DataBatch(edge_index=[2, 7758], x=[2182, 3], y=[64], batch=[2182], ptr=[65])
 - batch2: DataBatch(edge_index=[2, 8004], x=[2157, 3], y=[64], batch=[2157], ptr=[65])
 - batch3: DataBatch(edge_index=[2, 11238], x=[2989, 3], y=[64], batch=[2989], ptr=[65])
 - batch4: DataBatch(edge_index=[2, 8630], x=[2348, 3], y=[64], batch=[2348], ptr=[65])
 - batch5: DataBatch(edge_index=[2, 10772], x=[2845, 3], y=[64], batch=[2845], ptr=[65])
 - batch6: DataBatch(edge_index=[2, 9946], x=[2620, 3], y=[64], batch=[2620], ptr=[65])
 - batch7: DataBatch(edge_index=[2, 9348], x=[2501, 3], y=[64], batch=[2501], ptr=[65])
 - batch8: DataBatch(edge_index=[2, 9620], x=[2637, 3], y=[64], batch=[2637], ptr=[65])
 - batch9: DataBatch(edge_index=[2, 8704], x=[2412, 3], y=[64], batch=[2412], ptr=[65])
 - batch10: DataBatch(edge_index=[2, 9392], x=[2617, 3], y=[64], batch=[2617], ptr=[65])
 - batch11: DataBatch(edge_index=[2, 7568], x=[1981, 3], y=[64], batch=[1981], ptr=[65])
 - batch12: D

## 🧠 Summary: How GIN Works with Multiple Graphs (Batch Processing)

- Each batch contains several graphs.  
  Each graph has multiple nodes with feature vectors and its own edge structure.

- PyTorch Geometric combines all graphs in the batch into **one big graph**:
  - `x` → all node features stacked together  
  - `edge_index` → edges of all graphs with shifted node indices  
  - `batch` → vector indicating which node belongs to which graph

- A single **forward pass** is done for the entire batch (not one per graph).

- Inside each GIN layer:
$$
h_v^{(k)} = \mathrm{MLP}^{(k)}\Big((1+\epsilon)\,h_v^{(k-1)} + \mathrm{AGG}\big(\{h_u^{(k-1)} \mid u \in \mathcal{N}(v)\}\big)\Big)
$$

  - **MLP** = small neural network shared across all nodes and all graphs.  
  - **AGG** = aggregation function (`add`, `mean`, or `max`).  
  - **ε (eps)** controls the weight of the node’s own features (can be learnable if `train_eps=True`).

- 🔁 **Weight sharing**:
  - All nodes in all graphs share the same weights for each GIN layer.  
  - Only node features (`x`) and graph structures (`edge_index`) differ.  
  - During training, gradients from all graphs in the batch are averaged,  
    and the shared weights are updated **once per batch**.

- 🧩 After the GIN layers:
  - Node embeddings are aggregated (via `global_add_pool`, `global_mean_pool`, etc.)  
    to produce one embedding per graph.

- ⚙️ Then a final classifier (usually an MLP) predicts the label for each graph.

- ✅ Summary of updates:
  | Level | Shared Weights? | Updated When? |
  |--------|-----------------|----------------|
  | Nodes within a graph | ✅ Yes | N/A |
  | Graphs within a batch | ✅ Yes | Once per batch |
  | Between batches | ✅ Continue from previous step | After each optimizer step |

**In short:**  
GIN learns a single message-passing function (the same weights) that applies to every node of every graph, combines messages from neighbors, and updates weights only once per batch using the averaged gradient of all graphs.


In [5]:
import torch
from torch import nn
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F

from torch_geometric.nn import GINConv
from torch_geometric.nn import global_add_pool

In [6]:
def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

In [6]:
class GIN(nn.Module):
    def __init__(self, dim_h):
        super().__init__()

        self.gin1 = GINConv(Sequential(Linear(dataset.num_features, dim_h),
                                       BatchNorm1d(dim_h), ReLU(),
                                       Linear(dim_h, dim_h), ReLU()))
        self.gin2 = GINConv(Sequential(Linear(dim_h, dim_h),
                                       BatchNorm1d(dim_h), ReLU(),
                                       Linear(dim_h, dim_h), ReLU()))
        self.gin3 = GINConv(Sequential(Linear(dim_h, dim_h),
                                       BatchNorm1d(dim_h), ReLU(),
                                       Linear(dim_h, dim_h), ReLU()))

        self.lin1 = Linear(3 * dim_h, dim_h)
        self.lin2 = Linear(dim_h, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x1 = self.gin1(x, edge_index)
        x2 = self.gin2(x1, edge_index)
        x3 = self.gin3(x2, edge_index)

        h1 = global_add_pool(x1, batch)
        h2 = global_add_pool(x2, batch)
        h3 = global_add_pool(x3, batch)

        h = torch.cat([h1, h2, h3], dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h

    def fit(self, train_loader, val_loader, epochs):
        self.train()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()

        total_loss_list, acc_list, val_loss_list, val_acc_list = [], [], [], []

        for epoch in range(epochs):
            total_loss, acc, val_loss, val_acc = 0, 0, 0, 0

            for data in train_loader:
                optimizer.zero_grad()
                out = self(data.x, data.edge_index, data.batch)
                loss = criterion(out, data.y)
                total_loss += loss.item() / len(train_loader)
                acc += accuracy(out.argmax(dim=1), data.y) / len(train_loader)
                loss.backward()
                optimizer.step()

            with torch.no_grad():
                for data in val_loader:
                    out = self(data.x, data.edge_index, data.batch)
                    val_loss += criterion(out, data.y).item() / len(val_loader)
                    val_acc += accuracy(out.argmax(dim=1), data.y) / len(val_loader)

            total_loss_list.append(total_loss)
            acc_list.append(acc)
            val_loss_list.append(val_loss)
            val_acc_list.append(val_acc)

            print(f'Epoch {epoch+1}/{epochs}, '
                  f'Train Loss: {total_loss:.4f}, Train Acc: {acc:.4f}, '
                  f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        return total_loss_list, acc_list, val_loss_list, val_acc_list

In [4]:
# import inspect
# print(inspect.getsource(GINConv.forward))

    def forward(
        self,
        x: Union[Tensor, OptPairTensor],
        edge_index: Adj,
        size: Size = None,
    ) -> Tensor:

        if isinstance(x, Tensor):
            x = (x, x)

        # propagate_type: (x: OptPairTensor)
        out = self.propagate(edge_index, x=x, size=size)

        x_r = x[1]
        if x_r is not None:
            out = out + (1 + self.eps) * x_r

        return self.nn(out)

